TODO: Until we get real data, make synthetic data

#### Temporary: Synthetic Data

In [5]:
# Synthetic Data
import numpy as np
from sklearn.datasets import make_classification
X, y = make_classification(random_state=42)
data = np.concatenate((X, y.reshape(-1, 1)), axis = 1)

### Preprocessing

In [6]:
import preprocess
np.random.seed(42)
states = np.random.randint(low = 0, high = 1000000, size=(100,)) # numpy array with our 100 random states
data_splits = {}
for rst in states:
    data_splits[rst] = preprocess.preprocess(data, rst)

### Modeling

In [8]:
import modeling as models

model_res = {} # key as model type, then sub dictionary with rst as key and predicted classes & model itself ex: model_res["dt"][rst] --> (y_pred_dt, dt) 
model_res["dt"] = {}
model_res["rf"] = {}
model_res["gb"] = {}
model_res['xgb'] = {}
model_res["lgb"] = {}
model_res["et"] = {}
model_res["ab"] = {}
model_res["lr"] = {}
model_res["lr1"] = {}
model_res["lr2"] = {}
model_res["lre"] = {}
model_res["lsv"] = {}
model_res["nlsv"] = {}
model_res["knn"] = {}
model_res["lda"] = {}
model_res["gnb"] = {}
model_res["mlp"] = {}

for state in data_splits.keys():    
    model_res["dt"][state] = models.model_decisiontree(*data_splits[state]) # decision tree
    model_res["rf"][state] = models.model_randomforest(*data_splits[state]) # random forest
    model_res["gb"][state] = models.model_gradientboosting(*data_splits[state]) # grad boosting
    model_res['xgb'][state] = models.model_extremegb(*data_splits[state])
    model_res["lgb"][state] = models.model_lightgb(*data_splits[state])
    model_res["et"][state] = models.model_extratrees(*data_splits[state])
    model_res["ab"][state] = models.model_adaboost(*data_splits[state])
    model_res["lr"][state] = models.model_logisticregression(*data_splits[state])
    model_res["lr1"][state] = models.model_lassoregularization(*data_splits[state])
    model_res["lr2"][state] = models.model_ridgeRegularization(*data_splits[state])
    model_res["lre"][state] = models.model_elasticNetRegularization(*data_splits[state])
    model_res["lsv"][state] = models.model_linearSupportVector(*data_splits[state])
    model_res["nlsv"][state] = models.model_nonLinearSupportVector(*data_splits[state])
    model_res["knn"][state] = models.model_kNearestNeighbor(*data_splits[state])
    model_res["lda"][state] = models.model_linearDiscriminantAnalysis(*data_splits[state])
    model_res["gnb"][state] = models.model_gaussianNaiveBayes(*data_splits[state])
    model_res["mlp"][state] = models.model_multiLayerPerceptron(*data_splits[state])

c:\Users\maxep\OneDrive\Documents\Github Repo\SeniorProject\TheBioBros\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\maxep\OneDrive\Documents\Github Repo\SeniorProject\TheBioBros\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


ValueError: Solver lbfgs supports only 'l2' or None penalties, got l1 penalty.

### Evaluation

In [ ]:
# basic view for now to see some metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from metrics import getMetrics
for rst in data_splits.keys(): 
    print(f"Stats for {rst}")
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["dt"][rst][0], model_res["dt"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["rf"][rst][0], model_res["rf"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["gb"][rst][0], model_res["gb"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["xgb"][rst][0], model_res["xgb"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["lgb"][rst][0], model_res["lgb"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["et"][rst][0], model_res["et"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["ab"][rst][0], model_res["ab"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["lr"][rst][0], model_res["lr"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["lr1"][rst][0], model_res["lr1"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["lr2"][rst][0], model_res["lr2"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["lre"][rst][0], model_res["lre"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["lsv"][rst][0], model_res["lsv"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["nlsv"][rst][0], model_res["nlsv"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["knn"][rst][0], model_res["knn"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["lda"][rst][0], model_res["lda"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["gnb"][rst][0], model_res["gnb"][rst][1]))
    print(getMetrics(data_splits[rst][3], data_splits[rst][1], model_res["mlp"][rst][0], model_res["mlp"][rst][1]))


Stats for 121958
(np.float64(0.9230769230769231), 0.9, 0.7777777777777778, 1.0, 0.875)
(np.float64(0.989010989010989), 0.9, 0.7777777777777778, 1.0, 0.875)
(np.float64(0.9230769230769231), 0.9, 0.7777777777777778, 1.0, 0.875)


KeyError: 'lg'